# Data Cleaning

Section goals:
- continue cleaning data
- [abandoned?] extract cpu data
- store as cleaned csv files

API and data dictionary:
https://steamspy.com/api.php

Future ideas:
- pc requirements analysis over time
- picture analysis
- keyword/recommender analysis
- categories could make table in a database all on its own, perhaps in future
- for genres (and categories?) could create main genre, selected from list of key genres, allowing hybrids like action_adventure if contains both

In [1]:
# load extensions and magics

# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

# http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%version_information numpy, pandas

Software versions
Python 3.7.3 64bit [MSC v.1915 64 bit (AMD64)]
IPython 7.5.0
OS Windows 10 10.0.17763 SP0
numpy 1.16.3
pandas 0.24.2
Sat May 04 11:15:17 2019 GMT Summer Time

In [2]:
from ast import literal_eval

import pandas as pd

In [3]:
# customisations
pd.set_option("max_columns", 100)
# pd.reset_option("max_columns")
# pd.options.display.max_columns = 100

In [4]:
raw_steam_data = pd.read_csv('../data/steam_app_data.csv')

print('Rows:', raw_steam_data.shape[0])
print('Columns:', raw_steam_data.shape[1])
raw_steam_data.head()

Rows: 29235
Columns: 39


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 719, 'final': 7...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 1992},{'total': 0},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</s

# First Chunk of Processing

In [70]:
def process_null_cols(df, thresh=0.5):
    cutoff_count = len(df) * thresh

    null_counts = df.isnull().sum()
    null_columns = df.columns[null_counts > cutoff_count]
    # print('Dropped columns:', list(null_columns))

    return df.dropna(thresh=cutoff_count, axis=1)


def process_null_row(df, col):
    null_rows = df[df[col].isnull()].shape[0]
    # print("Dropped null '{}' rows:".format(col), null_rows)
    
    return df[df[col].notnull()]
    
    
def process_name(df):
    df = process_null_row(df, 'name')
    # print("Dropped 'none' name rows:", df[df['name'] == 'none'].shape[0])
    return df[df['name'] != 'none']


def process_age(df):
    # UK PEGI Age raings: 3, 7, 12, 16, 18
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]
    
    df['required_age'] = pd.cut(df['required_age'], cut_points, labels=label_values)
    return df


def process_release_date(df):
    df = df.copy()
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    # Only want released games
    df = df[df['coming_soon'] == False].copy()
    
    # probably more efficient way to handle this, but works for now
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    df.loc[df['date'] == '', 'date'] = None
    df['datetime'] = pd.to_datetime(df['date'], format='%d %b, %Y', errors='ignore')
    df['datetime'] = pd.to_datetime(df['date'], format='%b %Y', errors='ignore')
    df['release_date'] = pd.to_datetime(df['datetime'])
    
    df = df.drop(['coming_soon', 'date', 'datetime'], axis=1)
    return df


def convert_price(x):
    try:
        return literal_eval(x)
    except ValueError:
        return {'currency': 'GBP', 'initial': -1}

    
def process_price(df):
    df = df.copy()
    
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(convert_price)
    
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # Set free games to 0
    df.loc[df['is_free'], 'price'] = 0
    # 4 apps in USD or EUR. Set to -1 for now
    df.loc[df['currency'] != 'GBP', 'price'] = -1
    
    df = df.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    # store price in pounds, not pennies
    df['price'] /= 100
    
    return df


def is_english(x):
    if 'english' in x.lower():
        return True
    else:
        return False

    
def process_language(df):
    df = df.copy()
    
    # 13 rows with missing data, safe to drop
    df = df.dropna(subset=['supported_languages'])
    
    df['english'] = df['supported_languages'].apply(is_english)
    df = df.drop('supported_languages', axis=1)
    
    return df


steam_data = process_null_cols(raw_steam_data)
steam_data = process_null_row(steam_data, 'type')
steam_data = process_name(steam_data)
steam_data = steam_data.drop('type', axis=1)
steam_data = steam_data.drop_duplicates()
steam_data = process_age(steam_data)
steam_data = process_release_date(steam_data)
steam_data = process_price(steam_data)

# could be useful for future recommender or key-word analysis, but drop for now
steam_data = steam_data.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)

steam_data = process_language(steam_data)

# links to images. could be useful for future image analysis project
steam_data = steam_data.drop(['header_image', 'screenshots', 'background'], axis=1)

# nearly 10000 missing values, unlikely useful to this analysis
steam_data = steam_data.drop('website', axis=1)

first_chunk = steam_data.copy()
steam_data.head()

,name,steam_appid,required_age,pc_requirements,mac_requirements,linux_requirements,developers,publishers,packages,package_groups,platforms,categories,genres,movies,achievements,release_date,support_info,content_descriptors,price,english
0,Counter-Strike,10,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2000-11-01,"{'url': 'http://steamcommunity.com/app/10', 'e...","{'ids': [2, 5], 'notes': 'Includes intense vio...",7.19,True
1,Team Fortress Classic,20,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-04-01,"{'url': '', 'email': ''}","{'ids': [2, 5], 'notes': 'Includes intense vio...",3.99,True
2,Day of Defeat,30,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2003-05-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True
3,Deathmatch Classic,40,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2001-06-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True
4,Half-Life: Opposing Force,50,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],[32],"[{'name': 'default', 'title': 'Buy Half-Life: ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...","{'ids': [], 'notes': None}",3.99,True


# Second Chunk of Processing

In [71]:
def process_requirements(df):
    df = df.copy()
    # drop requirements for now, can return to later
    df = df.drop(['pc_requirements', 'mac_requirements', 'linux_requirements'], axis=1)
    return df


def parse_associates(x):
    if x:
        try:
            return literal_eval(x)
        except ValueError:
            return None
    else:
        return None


def get_other_associates(x):
    if x:
        if len(x) > 1:
            output = x[1]
            if len(x) > 2:
                for item in x[2:]:
                    output += ', '
                    output += item
            return output
        else:
            return None
    else:
        return None

    
def process_developers_and_publishers(df):
    df = df.copy()
    
    # handle developers column
    df['developers'] = df['developers'].apply(parse_associates)
    df['developer'] = df['developers'].apply(lambda x: x[0] if x else None)
    df['other_developers'] = df['developers'].apply(get_other_associates)
    
    # handle publishers column
    df['publishers'] = df['publishers'].apply(parse_associates)
    df['publisher'] = df['publishers'].apply(lambda x: x[0] if x else None)
    df['other_publishers'] = df['publishers'].apply(get_other_associates)
    
    df = df.drop(['developers', 'publishers'], axis=1)
    
    return df


def process_packages(df):
    df = df.copy()
    # not useful for now, drop package columns
    df = df.drop(['packages', 'package_groups'], axis=1)
    return df


def process_platforms(df):
    df = df.copy()
    df['platforms'] = df['platforms'].apply(lambda x: literal_eval(x))
    
    df['windows'] = df['platforms'].apply(lambda x: x['windows'])
    df['mac'] = df['platforms'].apply(lambda x: x['mac'])
    df['linux'] = df['platforms'].apply(lambda x: x['linux'])
    
    df = df.drop('platforms', axis=1)
    
    return df


def parse_categories(x):
    try:
        return {c['description']:1 for c in literal_eval(x)}
    except ValueError:
        return {}

    
def get_col_list(series):
    cols_dict = {}
    
    def create_col_dict(x):
        for item in x.keys():
            cols_dict[item] = 1
    
    series.apply(create_col_dict)
    
    return list(cols_dict.keys())
    
    
def process_categories(df):
    df = df[['steam_appid', 'categories']].copy()
    
    df['categories'] = df['categories'].apply(parse_categories)
    col_list = get_col_list(df['categories'])
    
    def set_category(x, col):
        if col in x.keys():
            return 1
        else:
            return 0
    
    for col in col_list:
        col_name = (col.lower()
                       .replace('-', '_')
                       .replace(' ', '_')
                       .replace('(', '')
                       .replace(')', '')
                       .replace('/', '_or_')
                   )
        df[col_name] = df['categories'].apply(set_category, args=(col,))
    
    df = df.drop('categories', axis=1)
    
    return df


steam_data = first_chunk.copy()

steam_data = process_requirements(steam_data)
steam_data = process_developers_and_publishers(steam_data)
steam_data = process_packages(steam_data)
steam_data = process_platforms(steam_data)

category_data = process_categories(steam_data)
# category_data.to_csv('../data/steam_category_data.csv', index=False)
steam_data = steam_data.drop('categories', axis=1)

second_chunk = steam_data.copy()
display(category_data.head())
steam_data.head()

,steam_appid,multi_player,online_multi_player,local_multi_player,valve_anti_cheat_enabled,single_player,steam_cloud,steam_achievements,steam_trading_cards,captions_available,partial_controller_support,includes_source_sdk,cross_platform_multiplayer,stats,commentary_available,includes_level_editor,steam_workshop,in_app_purchases,co_op,full_controller_support,steam_leaderboards,steamvr_collectibles,online_co_op,shared_or_split_screen,local_co_op,mmo,vr_support,mods,mods_require_hl2,steam_turn_notifications
0,10,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,name,steam_appid,required_age,genres,movies,achievements,release_date,support_info,content_descriptors,price,english,developer,other_developers,publisher,other_publishers,windows,mac,linux
0,Counter-Strike,10,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2000-11-01,"{'url': 'http://steamcommunity.com/app/10', 'e...","{'ids': [2, 5], 'notes': 'Includes intense vio...",7.19,True,Valve,None,Valve,None,True,True,True
1,Team Fortress Classic,20,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-04-01,"{'url': '', 'email': ''}","{'ids': [2, 5], 'notes': 'Includes intense vio...",3.99,True,Valve,None,Valve,None,True,True,True
2,Day of Defeat,30,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2003-05-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True,Valve,None,Valve,None,True,True,True
3,Deathmatch Classic,40,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2001-06-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True,Valve,None,Valve,None,True,True,True
4,Half-Life: Opposing Force,50,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...","{'ids': [], 'notes': None}",3.99,True,Gearbox Software,None,Valve,None,True,True,True


# Developing Third Chunk of Processing (wip)

## 1. Genres

In [38]:
temp_df = second_chunk.copy()
temp_df.head()

temp_df['genres'].isnull().sum() # 40
temp_df['genres'].value_counts()

# just work with non nulls for now, handle later
temp_df = temp_df[temp_df['genres'].notnull()]

genres = temp_df['genres'].copy()
genres = genres.apply(literal_eval)
genres = genres.apply(lambda x: [item['description'] for item in x])

def get_col_list(series):
    cols_dict = {}
    
    def create_col_dict(x):
        for item in x:
            cols_dict[item] = 1
    
    series.apply(create_col_dict)
    
    return list(cols_dict.keys())

col_list = get_col_list(genres)


def set_category(x, col):
    if col in x:
        return 1
    else:
        return 0

genre_df = pd.DataFrame()

for col in col_list:
    col_name = (col.lower()
                    .replace(' ', '_')
                    .replace('&', 'and')
               )
    genre_df[col_name] = genres.apply(set_category, args=(col,))
    
genre_df

,action,free_to_play,strategy,adventure,indie,rpg,animation_and_modeling,video_production,casual,simulation,racing,violent,massively_multiplayer,nudity,sports,early_access,gore,utilities,audio_production,design_and_illustration,web_publishing,photo_editing,education,software_training,sexual_content,game_development,accounting,documentary,tutorial
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Refactoring into functions and timing whether using dictionaries or lists is faster (dicts are slightly faster)

In [52]:
def parse_column(x):
    try:
        return [item['description'] for item in literal_eval(x)]
    except ValueError:
        return []


def get_col_list(series):
    cols_dict = {}
    
    def create_col_dict(x):
        for item in x:
            cols_dict[item] = 1
    
    series.apply(create_col_dict)
    
    return list(cols_dict.keys())


def set_category(x, col):
    if col in x:
        return 1
    else:
        return 0

    
def process_genres(df):
    df = df[['steam_appid', 'genres']].copy()
    
    df['genres'] = df['genres'].apply(parse_column)
    
    col_list = get_col_list(df['genres'])
    
    for col in col_list:
        col_name = (col.lower()
                        .replace(' ', '_')
                        .replace('&', 'and')
                   )
        df[col_name] = df['genres'].apply(set_category, args=(col,))

    df = df.drop('genres', axis=1)
    
    return df

import time
import statistics

times = []

for i in range(10):
    start = time.time()
    genre_df = process_genres(steam_data)
    end = time.time()
    
    print('\r', i, end='')
    
    times.append(end-start)

print('\rAverage time taken (list method):', statistics.mean(times))

genre_df.head()


Average time taken (list method): 1.501084804534912


,steam_appid,action,free_to_play,strategy,adventure,indie,rpg,animation_and_modeling,video_production,casual,simulation,racing,violent,massively_multiplayer,nudity,sports,early_access,gore,utilities,audio_production,design_and_illustration,web_publishing,photo_editing,education,software_training,sexual_content,game_development,accounting,documentary,tutorial
0,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
def parse_column(x):
    try:
        return {item['description']:1 for item in literal_eval(x)}
    except ValueError:
        return {}


def get_col_list(series):
    cols_dict = {}
    
    def create_col_dict(x):
        for item in x.keys():
            cols_dict[item] = 1
    
    series.apply(create_col_dict)
    
    return list(cols_dict.keys())


def set_category(x, col):
    if col in x.keys():
        return 1
    else:
        return 0

    
def process_genres(df):
    df = df[['steam_appid', 'genres']].copy()
    
    df['genres'] = df['genres'].apply(parse_column)
    
    col_list = get_col_list(df['genres'])
    
    for col in col_list:
        col_name = (col.lower()
                        .replace(' ', '_')
                        .replace('&', 'and')
                   )
        df[col_name] = df['genres'].apply(set_category, args=(col,))

    df = df.drop('genres', axis=1)
    
    return df

import time
import statistics

times = []

for i in range(10):
    start = time.time()
    genre_df = process_genres(steam_data)
    end = time.time()
    
    print('\r', i, end='')
    
    times.append(end-start)

print('\rAverage time taken (dict method):', statistics.mean(times))

genre_df.head()


Average time taken (dict method): 1.4422792434692382


,steam_appid,action,free_to_play,strategy,adventure,indie,rpg,animation_and_modeling,video_production,casual,simulation,racing,violent,massively_multiplayer,nudity,sports,early_access,gore,utilities,audio_production,design_and_illustration,web_publishing,photo_editing,education,software_training,sexual_content,game_development,accounting,documentary,tutorial
0,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 2. Movies, Achievements, Support Info, Content Descriptors

In [82]:
temp_df = second_chunk.copy()
temp_df.head()

temp_df['movies'].isnull().sum() # 1883
temp_df['movies'].value_counts() # looks like contains data for trailers

# can drop movies column, similar to screenshots
temp_df = temp_df.drop('movies', axis=1)
temp_df.head()

temp_df['achievements'].isnull().sum() # 1935
temp_df['achievements'].value_counts()
# contains information about individual achievements
# too specific for analysis and made redundant by steam_achievements data in categories
# also, about 11000 have 0 achievements, and 2000 are null
# will drop
temp_df = temp_df.drop('achievements', axis=1)
temp_df.head()

temp_df['support_info'].isnull().sum() # 0
temp_df['support_info'].value_counts()

# contains url and email for each app. Could separate out into own columns but won't be useful for analysis. Drop for now
temp_df = temp_df.drop('support_info', axis=1)
temp_df.head()

temp_df['content_descriptors'].isnull().sum() # 0
temp_df['content_descriptors'].value_counts()
# mostly just age-related warnings. Have age_rating column so this isn't useful

{'ids': [], 'notes': None}                                                                                                                                                                                                                                                                                                                                                                                                           26696
{'ids': [2, 5], 'notes': None}                                                                                                                                                                                                                                                                                                                                                                                                         444
{'ids': [1, 5], 'notes': None}                                                                                                                    

# Third Chunk of Processing

In [79]:
# TODO: refactor categories processing slightly to use these same functions
def parse_column(x):
    try:
        return {item['description']:1 for item in literal_eval(x)}
    except ValueError:
        return {}


def get_col_list(series):
    cols_dict = {}
    
    def create_col_dict(x):
        for item in x.keys():
            cols_dict[item] = 1
    
    series.apply(create_col_dict)
    
    return list(cols_dict.keys())


def set_category(x, col):
    if col in x.keys():
        return 1
    else:
        return 0

    
def process_genres(df):
    df = df[['steam_appid', 'genres']].copy()
    
    df['genres'] = df['genres'].apply(parse_column)
    
    col_list = get_col_list(df['genres'])
    
    for col in col_list:
        col_name = (col.lower()
                        .replace(' ', '_')
                        .replace('&', 'and')
                   )
        df[col_name] = df['genres'].apply(set_category, args=(col,))

    df = df.drop('genres', axis=1)
    
    return df


steam_data = second_chunk.copy()

genre_df = process_genres(steam_data)
genre_df.to_csv('../data/steam_genre_data.csv', index=False)
steam_data = steam_data.drop('genres', axis=1)

# drop movies column - just contains trailers (similar to screenshots)
# drop achievements column - too specific and redundant as have steam_achievements in categories
# drop support info column - not useful for analysis
steam_data = steam_data.drop(['movies', 'achievements', 'support_info', 'content_descriptors'], axis=1)

display(genre_df.head())
steam_data.head()

,steam_appid,action,free_to_play,strategy,adventure,indie,rpg,animation_and_modeling,video_production,casual,simulation,racing,violent,massively_multiplayer,nudity,sports,early_access,gore,utilities,audio_production,design_and_illustration,web_publishing,photo_editing,education,software_training,sexual_content,game_development,accounting,documentary,tutorial
0,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,name,steam_appid,required_age,release_date,price,english,developer,other_developers,publisher,other_publishers,windows,mac,linux
0,Counter-Strike,10,0,2000-11-01,7.19,True,Valve,None,Valve,None,True,True,True
1,Team Fortress Classic,20,0,1999-04-01,3.99,True,Valve,None,Valve,None,True,True,True
2,Day of Defeat,30,0,2003-05-01,3.99,True,Valve,None,Valve,None,True,True,True
3,Deathmatch Classic,40,0,2001-06-01,3.99,True,Valve,None,Valve,None,True,True,True
4,Half-Life: Opposing Force,50,0,1999-11-01,3.99,True,Gearbox Software,None,Valve,None,True,True,True


### Combining and exporting data frames

In [88]:
steam_data.to_csv('../data/steam_data_clean.csv', index=False)

steam_data_full = steam_data.merge(genre_df, how='left', on='steam_appid')
steam_data_full = steam_data_full.merge(category_data, how='left', on='steam_appid')

steam_data_full.to_csv('../data/steam_data_clean_full.csv', index=False)